In [26]:
import re
import uuid
import random
import hashlib
import operator
from datetime import datetime, timezone
from operator import itemgetter
from config import Config

In [3]:
class A(object):
    def __init__(self):
        self.account = None

In [4]:
a = A()
a.account = 1

In [6]:
lst1 = [a]
lst2 = [a]

In [7]:
lst1 == lst2

True

In [84]:
buyOrder0 = {
    'symbol': 'zsh/usdt',
    'sender':'ab',
    'price': 5.0,
    'send': 'usdt',
    'sendVol': 5.0,
    'get': 'zsh',
    'getVol': 1,
    'comissionAmount':2,
    'tradeTxId': uuid.uuid4().hex
}

buyOrder1 = {
    'symbol': 'zsh/usdt',
    'sender':'bb',
    'price': 10.0,
    'send': 'usdt',
    'sendVol': 10.0,
    'get': 'zsh',
    'getVol': 1,
    'comissionAmount':2,
    'tradeTxId': uuid.uuid4().hex
}

buyOrder2 = {
    'symbol': 'zsh/usdt',
    'sender':'cb',
    'price': 12.0,
    'send': 'usdt',
    'sendVol': 12.0,
    'get': 'zsh',
    'getVol': 1,
    'comissionAmount':2,
    'tradeTxId': uuid.uuid4().hex
}

buyOrder3 = {
    'symbol': 'zsh/usdt',
    'sender':'db',
    'price': 13.0,
    'send': 'usdt',
    'sendVol': 13.0,
    'get': 'zsh',
    'getVol': 1,
    'comissionAmount':2,
    'tradeTxId': uuid.uuid4().hex
}

buyOrder4 = {
    'symbol': 'zsh/usdt',
    'sender':'eb',
    'price': 14.0,
    'send': 'usdt',
    'sendVol': 14.0,
    'get': 'zsh',
    'getVol': 1,
    'comissionAmount':2,
    'tradeTxId': uuid.uuid4().hex
}

sellOrder = {
    'symbol': 'zsh/usdt',
    'sender':'as',
    'price': 9.0,
    'send': 'zsh',
    'sendVol': 3.0,
    'get': 'usdt',
    'getVol': 27.0,
    'comissionAmount':2,
    'tradeTxId': uuid.uuid4().hex
}

sellOrder1 = {
    'symbol': 'zsh/usdt',
    'sender':'bs',
    'price': 10.0,
    'send': 'zsh',
    'sendVol': 1.0,
    'get': 'usdt',
    'getVol': 9.0,
    'comissionAmount':2,
    'tradeTxId': uuid.uuid4().hex
}

oprderPool = [buyOrder0,buyOrder1,buyOrder2,sellOrder1,buyOrder3,buyOrder4,sellOrder]



In [87]:
def matchOrders(oprderPool, symbol='zsh/usdt'):
    
    tradebleAsset = re.split(r'/', symbol)[0]

    buyOrders = []
    for i in range(len(oprderPool)):
        if oprderPool[i]['get'] == tradebleAsset:
            buyOrders.append(oprderPool[i])

    sellOrders = []
    for j in range(len(oprderPool)):
        if oprderPool[j]['send'] == tradebleAsset:
            sellOrders.append(oprderPool[j])

    buyOrders.sort(key=operator.itemgetter('price'), reverse=True)
    sellOrders.sort(key=operator.itemgetter('price'))
  
    commonTxs = []
    toRemove = []

    c = 0
    i = 0
    volFullfillment = None
    matchedOrders[f'tx_{c}'] = {'sellId': [], 'buyId': []}

    for n in range(max(len(buyOrders), len(sellOrders))):

        if buyOrders[i]['price'] >= sellOrders[c]['price']:

            if buyOrders[i]['getVol'] == sellOrders[c]['sendVol']:

                # Create common tx
                sellSendAmount = sellOrders[c]['sendVol']
                buySendAmount = sellSendAmount*sellOrders[c]['price']

                tradeSellTx = {
                    'timestamp': datetime.now(timezone.utc).timestamp(),
                    'symbol': sellOrders[c]['symbol'],
                    'contract': sellOrders[c]['send'],
                    'sender': sellOrders[c]['sender'],
                    'recipient': buyOrders[i]['sender'],
                    'sendAmount': sellSendAmount,
                    'recieveAmount': buySendAmount,
                    'price': sellOrders[c]['price'],
                    'comissionAmount': (sellOrders[c]['comissionAmount']/100)*(sellSendAmount/sellOrders[c]['sendVol']),
                    'tradeTxId': sellOrders[c]['tradeTxId']
                }

                tradeBuyTx = {
                    'timestamp': datetime.now(timezone.utc).timestamp(),
                    'symbol': sellOrders[c]['symbol'],
                    'contract': re.split(r'/', sellOrders[c]['symbol'])[1],
                    'sender': buyOrders[i]['sender'],
                    'recipient': sellOrders[c]['sender'],
                    'sendAmount': buySendAmount,
                    'recieveAmount': sellSendAmount,
                    'price': sellOrders[c]['price'],
                    'comissionAmount': (buyOrders[i]['comissionAmount']/100)*(buySendAmount/buyOrders[i]['sendVol']),
                    'tradeTxId': buyOrders[i]['tradeTxId']
                }

                sellOrders[c]['sendVol'] -= sellSendAmount
                sellOrders[c]['getVol'] -= buySendAmount
                buyOrders[i]['sendVol'] -= buySendAmount
                buyOrders[i]['getVol'] -= sellSendAmount

                commonTxs.append(tradeSellTx)
                commonTxs.append(tradeBuyTx)

                # Form Delete List
                toRemove.append(sellOrders[c]['tradeTxId'])
                toRemove.append(buyOrders[i]['tradeTxId'])

                if (c+1 >= len(sellOrders)) or (i+1 >= len(buyOrders)):
                    print(f'====\nI broke on c: {c} and i: {i}')
                    break
                else:
                    c+=1
                    i+=1

                    

            elif buyOrders[i]['getVol'] > sellOrders[c]['sendVol']:

                # Create common tx
                sellSendAmount = sellOrders[c]['sendVol'] #min(sellOrders[c]['sendVol'], buyOrders[i]['getVol'])
                buySendAmount = sellSendAmount*sellOrders[c]['price']

                tradeSellTx = {
                    'timestamp': datetime.now(timezone.utc).timestamp(),
                    'symbol': sellOrders[c]['symbol'],
                    'contract': sellOrders[c]['send'],
                    'sender': sellOrders[c]['sender'],
                    'recipient': buyOrders[i]['sender'],
                    'sendAmount': sellSendAmount,
                    'recieveAmount': buySendAmount,
                    'price': sellOrders[c]['price'],
                    'comissionAmount': (sellOrders[c]['comissionAmount']/100)*(sellSendAmount/sellOrders[c]['sendVol']),
                    'tradeTxId': sellOrders[c]['tradeTxId']
                }

                tradeBuyTx = {
                    'timestamp': datetime.now(timezone.utc).timestamp(),
                    'symbol': sellOrders[c]['symbol'],
                    'contract': re.split(r'/', sellOrders[c]['symbol'])[1],
                    'sender': buyOrders[i]['sender'],
                    'recipient': sellOrders[c]['sender'],
                    'sendAmount': buySendAmount,
                    'recieveAmount': sellSendAmount,
                    'price': sellOrders[c]['price'],
                    'comissionAmount': (buyOrders[i]['comissionAmount']/100)*(buySendAmount/buyOrders[i]['sendVol']),
                    'tradeTxId': buyOrders[i]['tradeTxId']
                }

                sellOrders[c]['sendVol'] -= sellSendAmount
                sellOrders[c]['getVol'] -= buySendAmount
                buyOrders[i]['sendVol'] -= buySendAmount
                buyOrders[i]['getVol'] -= sellSendAmount

                commonTxs.append(tradeSellTx)
                commonTxs.append(tradeBuyTx)


                # Form Delete List
                toRemove.append(sellOrders[c]['tradeTxId'])

                if (c+1 >= len(sellOrders)) or (i+1 >= len(buyOrders)):
                    print(f'====\nI broke on c: {c} and i: {i}')
                    break
                else:
                    c+=1




            elif buyOrders[i]['getVol'] < sellOrders[c]['sendVol']:

                # Create common tx
                sellSendAmount = buyOrders[i]['getVol'] #min(sellOrders[c]['sendVol'], buyOrders[i]['getVol'])
                buySendAmount = sellSendAmount*sellOrders[c]['price']

                tradeSellTx = {
                    'timestamp': datetime.now(timezone.utc).timestamp(),
                    'symbol': sellOrders[c]['symbol'],
                    'contract': sellOrders[c]['send'],
                    'sender': sellOrders[c]['sender'],
                    'recipient': buyOrders[i]['sender'],
                    'sendAmount': sellSendAmount,
                    'recieveAmount': buySendAmount,
                    'price': sellOrders[c]['price'],
                    'comissionAmount': (sellOrders[c]['comissionAmount']/100)*(sellSendAmount/sellOrders[c]['sendVol']),
                    'tradeTxId': sellOrders[c]['tradeTxId']
                }

                tradeBuyTx = {
                    'timestamp': datetime.now(timezone.utc).timestamp(),
                    'symbol': sellOrders[c]['symbol'],
                    'contract': re.split(r'/', sellOrders[c]['symbol'])[1],
                    'sender': buyOrders[i]['sender'],
                    'recipient': sellOrders[c]['sender'],
                    'sendAmount': buySendAmount,
                    'recieveAmount': sellSendAmount,
                    'price': sellOrders[c]['price'],
                    'comissionAmount': (buyOrders[i]['comissionAmount']/100)*(buySendAmount/buyOrders[i]['sendVol']),
                    'tradeTxId': buyOrders[i]['tradeTxId']
                }

                sellOrders[c]['sendVol'] -= sellSendAmount
                sellOrders[c]['getVol'] -= buySendAmount
                buyOrders[i]['sendVol'] -= buySendAmount
                buyOrders[i]['getVol'] -= sellSendAmount

                commonTxs.append(tradeSellTx)
                commonTxs.append(tradeBuyTx)


                # Form Delete List
                toRemove.append(buyOrders[i]['tradeTxId'])

                if (c+1 >= len(sellOrders)) or (i+1 >= len(buyOrders)):
                    print(f'====\nI broke on c: {c} and i: {i}')
                    break
                else:
                    i+=1

    return commonTxs, toRemove


In [83]:
len(buyOrders)

5

In [88]:
commomTxs, toRemove = matchOrders(buyOrders, sellOrders)
commomTxs

====
I broke on c: 1 and i: 3


[{'timestamp': 1630495129.359011,
  'symbol': 'zsh/usdt',
  'contract': 'zsh',
  'sender': 'as',
  'recipient': 'eb',
  'sendAmount': 1,
  'recieveAmount': 9.0,
  'price': 9.0,
  'comissionAmount': 0.006666666666666666,
  'tradeTxId': '12cd826e53424c1cac91298c72da8885'},
 {'timestamp': 1630495129.359024,
  'symbol': 'zsh/usdt',
  'contract': 'usdt',
  'sender': 'eb',
  'recipient': 'as',
  'sendAmount': 9.0,
  'recieveAmount': 1,
  'price': 9.0,
  'comissionAmount': 0.012857142857142859,
  'tradeTxId': 'f4be81d4d6354c3bb51b4c3d435a098f'},
 {'timestamp': 1630495129.359043,
  'symbol': 'zsh/usdt',
  'contract': 'zsh',
  'sender': 'as',
  'recipient': 'db',
  'sendAmount': 1,
  'recieveAmount': 9.0,
  'price': 9.0,
  'comissionAmount': 0.01,
  'tradeTxId': '12cd826e53424c1cac91298c72da8885'},
 {'timestamp': 1630495129.359046,
  'symbol': 'zsh/usdt',
  'contract': 'usdt',
  'sender': 'db',
  'recipient': 'as',
  'sendAmount': 9.0,
  'recieveAmount': 1,
  'price': 9.0,
  'comissionAmount': 

In [89]:
toRemove

['f4be81d4d6354c3bb51b4c3d435a098f',
 '00314f0f007e4ee194286c5af3844001',
 '12cd826e53424c1cac91298c72da8885',
 '9e3f42f090c34bb296c962ab9e16b7e5',
 '1d4d0df261f34b66b783abc33b382516',
 '978cc11f522b49b6bb3d54e4b56e96b7']

In [90]:
removeDicts = [order for order in oprderPool if order['tradeTxId'] in toRemove]
removeDicts
    

[{'symbol': 'zsh/usdt',
  'sender': 'bb',
  'price': 10.0,
  'send': 'usdt',
  'sendVol': 0.0,
  'get': 'zsh',
  'getVol': 0.0,
  'comissionAmount': 2,
  'tradeTxId': '978cc11f522b49b6bb3d54e4b56e96b7'},
 {'symbol': 'zsh/usdt',
  'sender': 'cb',
  'price': 12.0,
  'send': 'usdt',
  'sendVol': 3.0,
  'get': 'zsh',
  'getVol': 0.0,
  'comissionAmount': 2,
  'tradeTxId': '9e3f42f090c34bb296c962ab9e16b7e5'},
 {'symbol': 'zsh/usdt',
  'sender': 'bs',
  'price': 10.0,
  'send': 'zsh',
  'sendVol': 0.0,
  'get': 'usdt',
  'getVol': -1.0,
  'comissionAmount': 2,
  'tradeTxId': '1d4d0df261f34b66b783abc33b382516'},
 {'symbol': 'zsh/usdt',
  'sender': 'db',
  'price': 13.0,
  'send': 'usdt',
  'sendVol': 4.0,
  'get': 'zsh',
  'getVol': 0,
  'comissionAmount': 2,
  'tradeTxId': '00314f0f007e4ee194286c5af3844001'},
 {'symbol': 'zsh/usdt',
  'sender': 'eb',
  'price': 14.0,
  'send': 'usdt',
  'sendVol': 5.0,
  'get': 'zsh',
  'getVol': 0,
  'comissionAmount': 2,
  'tradeTxId': 'f4be81d4d6354c3bb51

In [92]:
for order in removeDicts:
    oprderPool.remove(order)
    
oprderPool

[{'symbol': 'zsh/usdt',
  'sender': 'ab',
  'price': 5.0,
  'send': 'usdt',
  'sendVol': 5.0,
  'get': 'zsh',
  'getVol': 1,
  'comissionAmount': 2,
  'tradeTxId': '59182408e21d44de90fca07b077491da'}]